This is the Jupyter notebook that was used to obtain the Brownian simulations for the manuscript entitled "Taylor dispersion analysis in a viscoelastic fluid through arbitrarily shaped axisymmetric channels," an adaptation of the Jupyter notebook available at: https://github.com/jrchang612/Taylor_dispersion_arbitrary/
The adaptations basically consist of modifying the velocity profile and introducing the Weissenberg number, Wi, as a parameter, replacing the Newtonian fluid profile with a viscoelastic fluid that obeys the linear Phan-Thien-Tanner rheological model.

In [ ]:
import sys
import os
cwd = os.getcwd()
parent_wd = cwd.replace('/notebooks', '')
sys.path.insert(1, parent_wd)
sys.path.append("..")  # Esto sube un nivel desde notebooks/ a la raíz del proyecto
from taylor_utils.taylorViscoelastic import * 
output_path = parent_wd + '/data_output/'

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.rcParams['text.usetex'] = True
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp
from scipy.stats import norm
from matplotlib.gridspec import GridSpec

import pickle
def save_pickle(data, file_name):
    """
    Saves data as pickle format
    """
    with open(file_name, 'wb') as f:
        pickle.dump(data, f)
    return None

### Diverging Channel

In [ ]:
a0 = 1
Pe0 = 10
beta = 1E-3
Wi = 10
func_x = lambda x: a0 + beta*x

for seed in range(5):
    result =  simulation_var_cone_moment_initialVar(Pe0 = Pe0, Wi = Wi, func_x = func_x, Nt0 = 500, seed = seed, sigx2_0 = 10)
    save_pickle(result, output_path+'result_r_Pe0_'+str(Pe0)+'_Wi_'+str(Wi)+'_beta_'+str(beta)+'_moment_initVar10_seed_'+str(seed))
    print(seed)

### Converging Channel

In [ ]:
a0 = 1
Pe0 = 10
beta_f = -1E-3
func_x = lambda x: a0 + beta_f*x
Wi = 10

for seed in range(5):
    result =  simulation_var_cone_moment_initialVar(Pe0 = Pe0, Wi = Wi, func_x = func_x, Nt0 = 26, seed = seed, sigx2_0 = 10)
    save_pickle(result, output_path+'result_r_Pe0_'+str(Pe0)+'_Wi_'+str(Wi)+'_beta_'+str(beta_f)+'_moment_initVar10_seed_'+str(seed))
    print(seed)

### Sinusoidal Channel
Evaluated at different Peclet numbers

In [ ]:
func_x = lambda x: 1+0.2*np.sin(x/400)
Pe0 = 10
Wi=10
for seed in range(5):
    result =  simulation_var_cone_moment_initialVar(Pe0 = Pe0, Wi = Wi, func_x = func_x, Nt0 = 500, seed = seed, sigx2_0 = 10)
    save_pickle(result, output_path+'result_periodic2_Pe0_'+str(Pe0)+'_Wi_'+str(Wi)+'_moment_initVar10_seed_'+str(seed))
    print(seed)

In [ ]:
func_x = lambda x: 1+0.2*np.sin(x/400)
Pe0 = 100
Wi=10
for seed in range(5):
    result =  simulation_var_cone_moment_initialVar(Pe0 = Pe0, Wi = Wi, func_x = func_x, Nt0 = 500, seed = seed, sigx2_0 = 10)
    save_pickle(result, output_path+'result_periodic2_Pe0_'+str(Pe0)+'_Wi_'+str(Wi)+'_moment_initVar10_seed_'+str(seed))
    print(seed)

In [ ]:
func_x = lambda x: 1+0.2*np.sin(x/400)
Pe0 = 1000
Wi = 10
for seed in range(5):
    result =  simulation_var_cone_moment_initialVar(Pe0 = Pe0, Wi = Wi,  func_x = func_x, Nt0 = 500, seed = seed, sigx2_0 = 10)
    save_pickle(result, output_path+'result_periodic2_Pe0_'+str(Pe0)+'_Wi_'+str(Wi)+'_moment_initVar10_seed_'+str(seed))
    print(seed)

### Engineered channel for constant variance

In [ ]:
sigx2 = 300
U0 = 1
a0 = 1
D = 0.1
Wi = 10
def Unw(a): return ((9*(64*Wi/3)**2*(U0*a0**2)+(3)**0.5*(27*(64*Wi/3)**4*( U0*a0**2)**2+4*(64*Wi/3)**3*((a)**2)**3)**0.5)**(1/3)/((2)**(1/3)*3**(2/3)* (64*Wi/3))-((2/3)**(1/3)*(a)**2)/(9*(64*Wi/3)**2*(U0*a0**2)+(3)**0.5*(27*(64*Wi/3)**4*( U0*a0**2)**2+4*(64*Wi/3)**3*((a)**2)**3)**0.5)**(1/3))
def engineered(x, a): 
    unw_val = Unw(a)
    return (2*a**2*D/(U0*a0**2) + 1/24*U0*a0**2/D-(unw_val**6)*(256*Wi**2)/(24*U0*D*a0**2)-(a**2)*(unw_val**4)*(8*Wi)/(15*U0*D*a0**2))/(4*(sigx2)/a)

sol = solve_ivp(engineered, [0, 1700], [a0], t_eval=np.linspace(0, 1700, 1700))
sol2 = solve_ivp(engineered, [0, -1100], [a0], t_eval=np.linspace(0, -1100, 1100))
func_x = interp1d(np.hstack([sol2.t[1:], sol.t]), 
                  np.hstack([sol2.y.flatten()[1:], sol.y.flatten()]), 
                  kind='cubic',
                  bounds_error=False,
                  fill_value='extrapolate')

Pe0 = 10
for seed in range(5):
    result =  simulation_var_cone_moment_initialVar(Pe0 = Pe0, Wi = Wi, func_x = func_x, Nt0 = 120, seed = seed, sigx2_0 = 300, upper_bound = 1500)
    save_pickle(result, output_path+'result_constantSig2_Pe0_'+str(Pe0)+'_Wi_'+str(Wi)+'_moment_initVar300_seed_'+str(seed))
    print(seed)

### Engineered channel for sinusoidal variance

In [ ]:
sigx2 = 300
delt = 50
lamb = 600
U0 = 1
a0 = 1
D = 0.1
Wi = 10
def Unw(a): return ((9*(64*Wi/3)**2*(U0*a0**2)+(3)**0.5*(27*(64*Wi/3)**4*( U0*a0**2)**2+4*(64*Wi/3)**3*((a)**2)**3)**0.5)**(1/3)/((2)**(1/3)*3**(2/3)* (64*Wi/3))-((2/3)**(1/3)*(a)**2)/(9*(64*Wi/3)**2*(U0*a0**2)+(3)**0.5*(27*(64*Wi/3)**4*( U0*a0**2)**2+4*(64*Wi/3)**3*((a)**2)**3)**0.5)**(1/3))
def engineered(x, a): 
    unw_val = Unw(a)
    return (-2*np.pi*delt/lamb*np.cos(2*np.pi*x/lamb) 
                              + 2*a**2*D/(U0*a0**2) + 1/24*U0*a0**2/D-(unw_val**6)*(256*Wi**2)/(27*U0*D*a0**2)-(a**2)*(unw_val**4)*(8*Wi)/(15*U0*D*a0**2))/(4*(sigx2+delt*np.sin(2*np.pi*x/lamb))/a + a*(2*D/(U0*a0**2))*(2*np.pi*delt/lamb*np.cos(2*np.pi*x/lamb)))

sol = solve_ivp(engineered, [0, 1700], [a0], t_eval=np.linspace(0, 1700, 1700))
sol2 = solve_ivp(engineered, [0, -1100], [a0], t_eval=np.linspace(0, -1100, 1100))
func_x = interp1d(np.hstack([sol2.t[1:], sol.t]), 
                  np.hstack([sol2.y.flatten()[1:], sol.y.flatten()]), 
                  kind='cubic',
                  bounds_error=False,
                  fill_value='extrapolate')

Pe0 = 10
for seed in range(5):
    result =  simulation_var_cone_moment_initialVar(Pe0 = Pe0, Wi = Wi, func_x = func_x, Nt0 = 120, seed = seed, sigx2_0 = 303.97976805104173, upper_bound = 1500)
    save_pickle(result, output_path+'result_sineSig2_Pe0_'+str(Pe0)+'_Wi_'+str(Wi)+'_moment_initVar300_seed_'+str(seed))
    print(seed)

### Clean up to create summarize file (smaller file size)

In [ ]:
name_list = ['result_constantSig2_Pe0_10_Wi_10_moment_initVar300_seed_',
    'result_sineSig2_Pe0_10_Wi_10_moment_initVar300_seed_',
    'result_r_Pe0_10_Wi_10_beta_0.001_moment_initVar10_seed_',
           'result_r_Pe0_10_Wi_10_beta_-0.001_moment_initVar10_seed_', 
             'result_periodic2_Pe0_10_Wi_10_moment_initVar10_seed_',
             'result_periodic2_Pe0_100_Wi_10_moment_initVar10_seed_', 
             'result_periodic2_Pe0_1000_Wi_10_moment_initVar10_seed_']

for name in name_list:
    for i in range(5):
        result = pickle.load(open(output_path+name+str(i), 'rb'))
        del result['x']
        del result['T']
        del result['r']
        del result['theta']
        save_pickle(result, output_path+name+str(i)+'_summarized')
        print(i)

### Exporting Diverging Channel Data

In [ ]:
sns.set_context('talk')
sns.set_style('ticks')

name_head = 'result_r_Pe0_10_Wi_10_beta_0.001_moment_initVar10_seed_'
func_x = lambda x : 1 + 0.001*x
result = pickle.load(open(output_path+name_head+str(0), 'rb'))

j_range = [100, 4000, 8000, 12000, 16000]
cmap_list = ['Greys', 'Blues', 'Oranges', 'Greens', 'Reds', 'Purples']

for m, j2 in enumerate(j_range):
    j = np.argmin(np.abs(result['weighted_x'].flatten()[j2] - result['predicted_x_bar']))
    
    X = result['x'][:, j2]
    R = np.copy(result['r'][:, j2])
    R_plot = np.zeros_like(R)
    THETA = result['theta'][:, j2]
    loc_neg = THETA < 0
    R_plot = R[~loc_neg]
    Y = R_plot*np.sin(THETA[~loc_neg])
    X_plot = X[~loc_neg]
     
    xmin = np.min(X_plot)
    xmax = np.max(X_plot)
    i_plot = 200
    x_pdf = np.linspace(xmin, xmax, i_plot)
    alpha_all = (norm.pdf(x_pdf, result['predicted_x_bar'][j], 
                             np.sqrt(result['predicted_var_heuristic'][j]))/np.max(norm.pdf(x_pdf, result['predicted_x_bar'][j], 
                                                                                            np.sqrt(result['predicted_var_heuristic'][j])))
                 )
    for i in range(i_plot-1):
        pass  
    
time_scaled = result['T'] / (result['a0']**2/(2*result['D']))

weighted_x_mean = []
weighted_var_mean = []
predicted_x_bar_mean = []
predicted_var_mean = []
for i in range(5):
    result = pickle.load(open(output_path+name_head+str(i)+'_summarized', 'rb'))
    weighted_x_mean.append(result['weighted_x'].flatten())
    weighted_var_mean.append(result['weighted_var'].flatten())
    predicted_x_bar_mean.append(result['predicted_x_bar'])
    predicted_var_mean.append(result['predicted_var_heuristic'])
weighted_x_mean = np.mean(np.vstack(weighted_x_mean), axis = 0)
weighted_var_mean = np.mean(np.vstack(weighted_var_mean), axis = 0)
predicted_x_bar_mean = np.mean(np.vstack(predicted_x_bar_mean), axis = 0)
predicted_var_mean = np.mean(np.vstack(predicted_var_mean), axis = 0)

exportData = np.column_stack((weighted_x_mean, weighted_var_mean,predicted_x_bar_mean, predicted_var_mean))
np.savetxt('varianza_B_C_beta0.001_Wi10_Pe10.dat', exportData,  
           fmt='%.6f')

### Exporting Converging Channel Data

In [ ]:
sns.set_context('talk')
sns.set_style('ticks')

name_head = 'result_r_Pe0_10_Wi_10_beta_-0.001_moment_initVar10_seed_'
func_x = lambda x : 1 - 0.001*x
result = pickle.load(open(output_path+name_head+str(0), 'rb'))

j_range = [20, 200, 400, 600, 800]
cmap_list = ['Greys', 'Blues', 'Oranges', 'Greens', 'Reds', 'Purples']

for m, j2 in enumerate(j_range):
    j = np.nanargmin(np.abs(result['weighted_x'].flatten()[j2] - result['predicted_x_bar']))
    
    X = result['x'][:, j2]
    R = np.copy(result['r'][:, j2])
    R_plot = np.zeros_like(R)
    THETA = result['theta'][:, j2]
    loc_neg = THETA < 0
    R_plot = R[~loc_neg]
    Y = R_plot*np.sin(THETA[~loc_neg])
    X_plot = X[~loc_neg]
     
    xmin = np.min(X_plot)
    xmax = np.max(X_plot)
    i_plot = 200
    x_pdf = np.linspace(xmin, xmax, i_plot)
    alpha_all = (norm.pdf(x_pdf, result['predicted_x_bar'][j], 
                             np.sqrt(result['predicted_var_heuristic'][j]))/np.nanmax(norm.pdf(x_pdf, result['predicted_x_bar'][j], 
                                                                                            np.sqrt(result['predicted_var_heuristic'][j])))
                 )
    for i in range(i_plot-1):
        pass  

weighted_x_mean = []
weighted_var_mean = []
predicted_x_bar_mean = []
predicted_var_mean = []
for i in range(5):
    result = pickle.load(open(output_path+name_head+str(i)+'_summarized', 'rb'))
    weighted_x_mean.append(result['weighted_x'].flatten())
    weighted_var_mean.append(result['weighted_var'].flatten())
    predicted_x_bar_mean.append(result['predicted_x_bar'])
    predicted_var_mean.append(result['predicted_var_heuristic'])
weighted_x_mean = np.mean(np.vstack(weighted_x_mean), axis = 0)
weighted_var_mean = np.mean(np.vstack(weighted_var_mean), axis = 0)
predicted_x_bar_mean = np.mean(np.vstack(predicted_x_bar_mean), axis = 0)
predicted_var_mean = np.mean(np.vstack(predicted_var_mean), axis = 0)

exportData = np.column_stack((weighted_x_mean, weighted_var_mean,predicted_x_bar_mean, predicted_var_mean))
np.savetxt('varianza_B_C_beta-0.001_Wi10_Pe10.dat', exportData,  
           fmt='%.6f')

### Exporting Sinusoidal Channel Data

In [ ]:
sns.set_context('talk')
sns.set_style('ticks')
name_head = 'result_periodic2_Pe0_10_Wi_10_moment_initVar10_seed_'
func_x = lambda x: 1+0.2*np.sin(x/400)

result = pickle.load(open(output_path+name_head+str(0), 'rb'))

j_range = [100, 4000, 8000, 12000, 16000]
cmap_list = ['Greys', 'Blues', 'Oranges', 'Greens', 'Reds', 'Purples']

for m, j2 in enumerate(j_range):
    j = np.argmin(np.abs(result['weighted_x'].flatten()[j2] - result['predicted_x_bar']))
    
    X = result['x'][:, j2]
    R = np.copy(result['r'][:, j2])
    R_plot = np.zeros_like(R)
    THETA = result['theta'][:, j2]
    loc_neg = THETA < 0
    R_plot = R[~loc_neg]
    Y = R_plot*np.sin(THETA[~loc_neg])
    X_plot = X[~loc_neg]
        
    xmin = np.min(X_plot)
    xmax = np.max(X_plot)
    i_plot = 200
    x_pdf = np.linspace(xmin, xmax, i_plot)
    alpha_all = (norm.pdf(x_pdf, result['predicted_x_bar'][j], 
                             np.sqrt(result['predicted_var_heuristic'][j]))/np.max(norm.pdf(x_pdf, result['predicted_x_bar'][j], 
                                                                                            np.sqrt(result['predicted_var_heuristic'][j])))
                )
    for i in range(i_plot-1):
        pass

time_scaled = result['T'] / (result['a0']**2/(2*result['D']))
weighted_x_mean = []
weighted_var_mean = []
predicted_x_bar_mean = []
predicted_var_mean = []
for i in range(5):
    result = pickle.load(open(output_path+name_head+str(i)+'_summarized', 'rb'))
    weighted_x_mean.append(result['weighted_x'].flatten())
    weighted_var_mean.append(result['weighted_var'].flatten())
    predicted_x_bar_mean.append(result['predicted_x_bar'])
    predicted_var_mean.append(result['predicted_var_heuristic'])
weighted_x_mean = np.mean(np.vstack(weighted_x_mean), axis = 0)
weighted_var_mean = np.mean(np.vstack(weighted_var_mean), axis = 0)
predicted_x_bar_mean = np.mean(np.vstack(predicted_x_bar_mean), axis = 0)
predicted_var_mean = np.mean(np.vstack(predicted_var_mean), axis = 0)

a = func_x(predicted_x_bar_mean)
W=10
U = result['U0']*result['a0']**2/func_x(predicted_x_bar_mean)**2
Un = (9*(64*W/3)**2*(result['U0']*result['a0']**2)+(3)**0.5*(27*(64*W/3)**4*( result['U0']*result['a0']**2)**2+4*(64*W/3)**3*((func_x(predicted_x_bar_mean))**2)**3)**0.5)**(1/3)/((2)**(1/3)*3**(2/3)* (64*W/3))-((2/3)**(1/3)*(func_x(predicted_x_bar_mean))**2)/(9*(64*W/3)**2*(result['U0']*result['a0']**2)+(3)**0.5*(27*(64*W/3)**4*( result['U0']*result['a0']**2)**2+4*(64*W/3)**3*((func_x(predicted_x_bar_mean))**2)**3)**0.5)**(1/3)
Uv = U-Un
beta_of_x = interp1d(result['x_range'], result['beta'], kind='cubic')

Deff= 1 + 1/48*U**2*a**2/result['D']**2-1/96*Uv**2*a**2/result['D']**2-1/80*Uv*Un*a**2/result['D']**2

exportData = np.column_stack((weighted_x_mean, weighted_var_mean,predicted_x_bar_mean, predicted_var_mean, Deff, time_scaled ))
np.savetxt('varianza_B_C_periodic_Wi10_Pe10.dat', exportData,  
           fmt='%.6f')

In [ ]:
sns.set_context('talk')
sns.set_style('ticks')
name_head = 'result_periodic2_Pe0_100_Wi_10_moment_initVar10_seed_'
func_x = lambda x: 1+0.2*np.sin(x/400)

result = pickle.load(open(output_path+name_head+str(0), 'rb'))

j_range = [100, 4000, 8000, 12000, 16000]
cmap_list = ['Greys', 'Blues', 'Oranges', 'Greens', 'Reds', 'Purples']

for m, j2 in enumerate(j_range):
    j = np.argmin(np.abs(result['weighted_x'].flatten()[j2] - result['predicted_x_bar']))
    
    X = result['x'][:, j2]
    R = np.copy(result['r'][:, j2])
    R_plot = np.zeros_like(R)
    THETA = result['theta'][:, j2]
    loc_neg = THETA < 0
    R_plot = R[~loc_neg]
    Y = R_plot*np.sin(THETA[~loc_neg])
    X_plot = X[~loc_neg]
        
    xmin = np.min(X_plot)
    xmax = np.max(X_plot)
    i_plot = 200
    x_pdf = np.linspace(xmin, xmax, i_plot)
    alpha_all = (norm.pdf(x_pdf, result['predicted_x_bar'][j], 
                             np.sqrt(result['predicted_var_heuristic'][j]))/np.max(norm.pdf(x_pdf, result['predicted_x_bar'][j], 
                                                                                            np.sqrt(result['predicted_var_heuristic'][j])))
                )
    for i in range(i_plot-1):
        pass

time_scaled = result['T'] / (result['a0']**2/(2*result['D']))
weighted_x_mean = []
weighted_var_mean = []
predicted_x_bar_mean = []
predicted_var_mean = []
for i in range(5):
    result = pickle.load(open(output_path+name_head+str(i)+'_summarized', 'rb'))
    weighted_x_mean.append(result['weighted_x'].flatten())
    weighted_var_mean.append(result['weighted_var'].flatten())
    predicted_x_bar_mean.append(result['predicted_x_bar'])
    predicted_var_mean.append(result['predicted_var_heuristic'])
weighted_x_mean = np.mean(np.vstack(weighted_x_mean), axis = 0)
weighted_var_mean = np.mean(np.vstack(weighted_var_mean), axis = 0)
predicted_x_bar_mean = np.mean(np.vstack(predicted_x_bar_mean), axis = 0)
predicted_var_mean = np.mean(np.vstack(predicted_var_mean), axis = 0)

a = func_x(predicted_x_bar_mean)
W=10
U = result['U0']*result['a0']**2/func_x(predicted_x_bar_mean)**2
Un = (9*(64*W/3)**2*(result['U0']*result['a0']**2)+(3)**0.5*(27*(64*W/3)**4*( result['U0']*result['a0']**2)**2+4*(64*W/3)**3*((func_x(predicted_x_bar_mean))**2)**3)**0.5)**(1/3)/((2)**(1/3)*3**(2/3)* (64*W/3))-((2/3)**(1/3)*(func_x(predicted_x_bar_mean))**2)/(9*(64*W/3)**2*(result['U0']*result['a0']**2)+(3)**0.5*(27*(64*W/3)**4*( result['U0']*result['a0']**2)**2+4*(64*W/3)**3*((func_x(predicted_x_bar_mean))**2)**3)**0.5)**(1/3)
Uv = U-Un
beta_of_x = interp1d(result['x_range'], result['beta'], kind='cubic')

Deff= 1 + 1/48*U**2*a**2/result['D']**2-1/96*Uv**2*a**2/result['D']**2-1/80*Uv*Un*a**2/result['D']**2

exportData = np.column_stack((weighted_x_mean, weighted_var_mean,predicted_x_bar_mean, predicted_var_mean, Deff, time_scaled ))
np.savetxt('varianza_B_C_periodic_Wi10_Pe100.dat', exportData,  
           fmt='%.6f')

In [ ]:
sns.set_context('talk')
sns.set_style('ticks')
name_head = 'result_periodic2_Pe0_1000_Wi_10_moment_initVar10_seed_'
func_x = lambda x: 1+0.2*np.sin(x/400)

result = pickle.load(open(output_path+name_head+str(0), 'rb'))

j_range = [100, 4000, 8000, 12000, 16000]
cmap_list = ['Greys', 'Blues', 'Oranges', 'Greens', 'Reds', 'Purples']

for m, j2 in enumerate(j_range):
    j = np.argmin(np.abs(result['weighted_x'].flatten()[j2] - result['predicted_x_bar']))
    
    X = result['x'][:, j2]
    R = np.copy(result['r'][:, j2])
    R_plot = np.zeros_like(R)
    THETA = result['theta'][:, j2]
    loc_neg = THETA < 0
    R_plot = R[~loc_neg]
    Y = R_plot*np.sin(THETA[~loc_neg])
    X_plot = X[~loc_neg]
        
    xmin = np.min(X_plot)
    xmax = np.max(X_plot)
    i_plot = 200
    x_pdf = np.linspace(xmin, xmax, i_plot)
    alpha_all = (norm.pdf(x_pdf, result['predicted_x_bar'][j], 
                             np.sqrt(result['predicted_var_heuristic'][j]))/np.max(norm.pdf(x_pdf, result['predicted_x_bar'][j], 
                                                                                            np.sqrt(result['predicted_var_heuristic'][j])))
                )
    for i in range(i_plot-1):
        pass

time_scaled = result['T'] / (result['a0']**2/(2*result['D']))
weighted_x_mean = []
weighted_var_mean = []
predicted_x_bar_mean = []
predicted_var_mean = []
for i in range(5):
    result = pickle.load(open(output_path+name_head+str(i)+'_summarized', 'rb'))
    weighted_x_mean.append(result['weighted_x'].flatten())
    weighted_var_mean.append(result['weighted_var'].flatten())
    predicted_x_bar_mean.append(result['predicted_x_bar'])
    predicted_var_mean.append(result['predicted_var_heuristic'])
weighted_x_mean = np.mean(np.vstack(weighted_x_mean), axis = 0)
weighted_var_mean = np.mean(np.vstack(weighted_var_mean), axis = 0)
predicted_x_bar_mean = np.mean(np.vstack(predicted_x_bar_mean), axis = 0)
predicted_var_mean = np.mean(np.vstack(predicted_var_mean), axis = 0)

a = func_x(predicted_x_bar_mean)
W=10
U = result['U0']*result['a0']**2/func_x(predicted_x_bar_mean)**2
Un = (9*(64*W/3)**2*(result['U0']*result['a0']**2)+(3)**0.5*(27*(64*W/3)**4*( result['U0']*result['a0']**2)**2+4*(64*W/3)**3*((func_x(predicted_x_bar_mean))**2)**3)**0.5)**(1/3)/((2)**(1/3)*3**(2/3)* (64*W/3))-((2/3)**(1/3)*(func_x(predicted_x_bar_mean))**2)/(9*(64*W/3)**2*(result['U0']*result['a0']**2)+(3)**0.5*(27*(64*W/3)**4*( result['U0']*result['a0']**2)**2+4*(64*W/3)**3*((func_x(predicted_x_bar_mean))**2)**3)**0.5)**(1/3)
Uv = U-Un
beta_of_x = interp1d(result['x_range'], result['beta'], kind='cubic')

Deff= 1 + 1/48*U**2*a**2/result['D']**2-1/96*Uv**2*a**2/result['D']**2-1/80*Uv*Un*a**2/result['D']**2

exportData = np.column_stack((weighted_x_mean, weighted_var_mean,predicted_x_bar_mean, predicted_var_mean, Deff, time_scaled ))
np.savetxt('varianza_B_C_periodic_Wi10_Pe1000.dat', exportData,  
           fmt='%.6f')

### Exporting Engineered channel for constant variance Data

In [ ]:
sns.set_context('talk')
sns.set_style('ticks')
name_head = 'result_constantSig2_Pe0_10_Wi_10_moment_initVar300_seed_'

result = pickle.load(open(output_path+name_head+str(0), 'rb'))

sigx2 = 300
delt = 50
U0 = 1
a0 = 1
D = 0.1
Wi = 10
def Unw(a): return ((9*(64*Wi/3)**2*(U0*a0**2)+(3)**0.5*(27*(64*Wi/3)**4*( U0*a0**2)**2+4*(64*Wi/3)**3*((a)**2)**3)**0.5)**(1/3)/((2)**(1/3)*3**(2/3)* (64*Wi/3))-((2/3)**(1/3)*(a)**2)/(9*(64*Wi/3)**2*(U0*a0**2)+(3)**0.5*(27*(64*Wi/3)**4*( U0*a0**2)**2+4*(64*Wi/3)**3*((a)**2)**3)**0.5)**(1/3))
def engineered(x, a): 
    unw_val = Unw(a)
    return (2*a**2*D/(U0*a0**2) + 1/24*U0*a0**2/D-(unw_val**6)*(256*Wi**2)/(27*U0*D*a0**2)-(a**2)*(unw_val**4)*(8*Wi)/(15*U0*D*a0**2))/(4*(sigx2)/a)

sol = solve_ivp(engineered, [0, 1700], [a0], t_eval=np.linspace(0, 1700, 1700))
sol2 = solve_ivp(engineered, [0, -1100], [a0], t_eval=np.linspace(0, -1100, 1100))
func_x = interp1d(np.hstack([sol2.t[1:], sol.t]), 
                  np.hstack([sol2.y.flatten()[1:], sol.y.flatten()]), 
                  kind='cubic',
                  bounds_error=False,
                  fill_value='extrapolate')

x_export = np.linspace(-100, 1000, 1000)
a_values = func_x(x_export)
var_target = sigx2 * np.ones_like(x_export)

weighted_x_all = []
weighted_var_all = []
predicted_x_all = []
predicted_var_all = []

for i in range(5):
    result_data = pickle.load(open(output_path+name_head+str(i)+'_summarized', 'rb'))
    weighted_x_all.append(result_data['weighted_x'].flatten())
    weighted_var_all.append(result_data['weighted_var'].flatten())
    predicted_x_all.append(result_data['predicted_x_bar'].flatten())
    predicted_var_all.append(result_data['predicted_var_heuristic'].flatten())

weighted_x_mean = np.mean(np.vstack(weighted_x_all), axis=0)
weighted_var_mean = np.mean(np.vstack(weighted_var_all), axis=0)
predicted_x_mean = np.mean(np.vstack(predicted_x_all), axis=0)
predicted_var_mean = np.mean(np.vstack(predicted_var_all), axis=0)

var_brownian_interp = np.interp(x_export, weighted_x_mean, weighted_var_mean)
var_calculated_interp = np.interp(x_export, predicted_x_mean, predicted_var_mean)

export_data = np.column_stack((
    x_export,
    a_values,
    var_calculated_interp,
    var_brownian_interp,
    var_target
))

np.savetxt('dats_sim_Enginnering_Constant.dat', export_data, 
           fmt='%.6f')


### Exporting Engineered channel for sinusoidal variance Data

In [ ]:
sns.set_context('talk')
sns.set_style('ticks')
name_head = 'result_sineSig2_Pe0_10_Wi_10_moment_initVar300_seed_'

result = pickle.load(open(output_path+name_head+str(0), 'rb'))

sigx2 = 300
delt = 50
lamb = 600
U0 = 1
a0 = 1
D = 0.1
Wi = 10
def Unw(a): return ((9*(64*Wi/3)**2*(U0*a0**2)+(3)**0.5*(27*(64*Wi/3)**4*( U0*a0**2)**2+4*(64*Wi/3)**3*((a)**2)**3)**0.5)**(1/3)/((2)**(1/3)*3**(2/3)* (64*Wi/3))-((2/3)**(1/3)*(a)**2)/(9*(64*Wi/3)**2*(U0*a0**2)+(3)**0.5*(27*(64*Wi/3)**4*( U0*a0**2)**2+4*(64*Wi/3)**3*((a)**2)**3)**0.5)**(1/3))
def engineered(x, a): 
    unw_val = Unw(a)
    return (-2*np.pi*delt/lamb*np.cos(2*np.pi*x/lamb) 
                              + 2*a**2*D/(U0*a0**2) + 1/24*U0*a0**2/D-(unw_val**6)*(256*Wi**2)/(24*U0*D*a0**2)-(a**2)*(unw_val**4)*(8*Wi)/(15*U0*D*a0**2))/(4*(sigx2+delt*np.sin(2*np.pi*x/lamb))/a + a*(2*D/(U0*a0**2))*(2*np.pi*delt/lamb*np.cos(2*np.pi*x/lamb)))

sol = solve_ivp(engineered, [0, 1700], [a0], t_eval=np.linspace(0, 1700, 1700))
sol2 = solve_ivp(engineered, [0, -1100], [a0], t_eval=np.linspace(0, -1100, 1100))
func_x = interp1d(np.hstack([sol2.t[1:], sol.t]), 
                  np.hstack([sol2.y.flatten()[1:], sol.y.flatten()]), 
                  kind='cubic',
                  bounds_error=False,
                  fill_value='extrapolate')

x_export = np.linspace(-100, 1000, 1000)
a_values = func_x(x_export)
var_target = sigx2 + delt * np.sin(2 * np.pi * x_export / lamb)

weighted_x_all = []
weighted_var_all = []
predicted_x_all = []
predicted_var_all = []

for i in range(5):
    result_data = pickle.load(open(output_path+name_head+str(i)+'_summarized', 'rb'))
    weighted_x_all.append(result_data['weighted_x'].flatten())
    weighted_var_all.append(result_data['weighted_var'].flatten())
    predicted_x_all.append(result_data['predicted_x_bar'].flatten())
    predicted_var_all.append(result_data['predicted_var_heuristic'].flatten())

weighted_x_mean = np.mean(np.vstack(weighted_x_all), axis=0)
weighted_var_mean = np.mean(np.vstack(weighted_var_all), axis=0)
predicted_x_mean = np.mean(np.vstack(predicted_x_all), axis=0)
predicted_var_mean = np.mean(np.vstack(predicted_var_all), axis=0)

var_brownian_interp = np.interp(x_export, weighted_x_mean, weighted_var_mean)
var_calculated_interp = np.interp(x_export, predicted_x_mean, predicted_var_mean)

export_data = np.column_stack((
    x_export,
    a_values,
    var_calculated_interp,
    var_brownian_interp,
    var_target
))

np.savetxt('dats_sim_Enginnering_Sinusoidal.dat', export_data, 
           fmt='%.6f')